# Principle Component Analysis

PCA is one of the fundamental methods in transforming data and especially dimension reduction.

## Variance and covariance matrix
Suppose $X = (X_1, X_2, \cdots, X_n)$ is a p-dimentional random variable, its variance (in high dimension form) is the covariance matrix 
$$\textrm{var}[X] = \Sigma = \textrm{E}\left[(X-\textrm{E}(X))(X-\textrm{E}(X)^T)\right], $$
where element on the diagonal are $\textrm{var}(X_i)$, other elements are covariance $\textrm{cov}(X_i, X_j).$

Suppose we have a transformed version $Z=AX$, its variance 
$$
\begin{align}
\textrm{var}(Z) =& \textrm{E}\left[(Z-\textrm{E}(Z))(Z-\textrm{E}(Z)^T)\right] \\
=& \textrm{E}\left[ (AX-A\mu_X)(AX-A\mu_X)^T \right] \\
=& \textrm{E}\left[ A(X-\mu_X)(X-\mu_X)^T A^T \right] = A\textrm{var}(X) A^T.
\end{align}$$

# PCA and dimension reduction

The essense of PCA is to find a transformation of $X$ such that in a new corridinate system $Q$, the covariance (and therefore correlation) become zero between any two dimensions. 

Let $X = QZ$, which says $Z$ is the coordinate of $X$ in the space of $Q$. For $Z = Q^T X$, we want to find the $Q$ so that its covariance matrix can be writen as a diagonal matrix $\Lambda$, so that the covariance and correlations between any two dimensions are removed in this new system.
$$\textrm{var}(Z) = Q^T \Sigma Q = \begin{bmatrix} \lambda_1 & & & \\ & \lambda_2 & & \\ & & \ddots & \\ & & & \lambda_p  \end{bmatrix} = \Lambda,$$
where $\Sigma$ is the covariance matrix of $X$.

Reorganize we have $\Sigma = Q \Lambda Q^T$, therefore $\Sigma Q = Q\Lambda$. It is clear that column vectors in $Q$ and diagonal elementsin $\Lambda$ are eigenvectors and eigenvalues of $\Sigma$. Hence the eigen decomposition of the covariance matrix $\Sigma$ of $X$ gives us the priciple directions $q_i$ and the variability $\lambda_i$ of data along the corresponding direction.

If top $k$ basis in $Q$ are selected according to $\lambda_i$ from biggest to smallest, we captured the top $k$ pricinple directions of data along which the variability are most significant. The choice of $k$ in practice can be choosen by keeping the reconstruction error of data in an acceptable range.


## Power method

### Vector form

For a vector $v$, let $u$ be the transformed coordinate in system $Q$, i.e. $v = Qu$.
$$
\Sigma v = Q\Lambda Q^T Q u = Q \begin{bmatrix} \lambda_1 & & & \\ & \lambda_2 & & \\ & & \ddots & \\ & & & \lambda_p  \end{bmatrix} \begin{bmatrix} u_1 \\ u_2 \\ \vdots \\ u_p  \end{bmatrix} = Q \begin{bmatrix} \lambda_1 u_1 \\ \lambda_2 u_2 \\ \vdots \\ \lambda_p u_p  \end{bmatrix}.
$$
This says the vector $\Sigma v$ becomes $(\lambda_1 u_1, \lambda_2 u_2, \cdots, \lambda_p u_p)^T$ in system $Q$.

If we repeat this process $n$ times, 
$$v \xrightarrow{\Sigma^n} (\lambda_1^n u_1, \lambda_2^n u_2, \cdots, \lambda_p^n u_p )^T. $$

Suppose $\lambda_1$ has the greatest magnitude, this procedure will converge to $v = (1, 0, \cdots, 0)$ in the space of $Q$, i.e. $v = q_1$. 

* Repeat the following steps: 
    - Compute normalized vector $\tilde{v} = \dfrac{v}{|v|}$.
    - Update $v = \Sigma \tilde{v}$.

To get $q_2$ using this method, we choose a vector $v \perp q_1$ that is perpendicular to $q_1$. To get $q_3$, choose $v$ to be perpendicular to both $q_1$ and $q_2$, i.e. $v \perp q_1$ and $v \perp q_2$. So on and so forth, we can get all the vectors in $Q$.

### Matrix form

Suppose initially we have a random matrix $V = (v_1, v_2, \cdots, v_p)$, whose orthogonalized version is $\tilde{V}=(\tilde{v}_1, \tilde{v}_2, \cdots, \tilde{v}_p)$ where each of these column vectors is a unit vector and every two of them are orthogonal to each other.

Power method is a iterative algorithm to compute eigen decomposition.

* Repeat the following steps: 
    - Compute $\tilde{V}$, the orthogonalized $V$.
    - Update $V = \Sigma \tilde{V}$.
    
In the following implementation, we use QR decomposition to perform orthogonalization.

In [1]:
import numpy as np

def eigen_qr(A):
    """An implementation of eigen decomposition using QR decompostion.
    
    Args:
        A: A square matrix.
        
    Returns:
        D: Eigen values. 
        V: A matrix whose columns are eigen vectors.
    """
    T = 1000
    r, c = A.shape

    V = np.random.random_sample((r, r))

    for i in range(T):
        Q, _ = np.linalg.qr(V)   # orthogonalize V
        V = np.dot(A, Q)         # update V

    Q, R = np.linalg.qr(V)

    return R.diagonal(), Q

In [2]:
# fix the random seed.
np.random.seed(1)

n = 100
p = 5
X = np.random.random_sample((n, p))
A = np.dot(X.T, X)

# Use the eigen_qr implemented above.
D, V = eigen_qr(A)
print 'eigen values  [ours]:', D.round(6)
print 'eigen vectors [ours]:\n',V.round(6)

# Compare the result with the numpy calculation.
print '=' * 10
eigen_value_gt, eigen_vector_gt = np.linalg.eig(A)
print 'eigen values  [numpy]:', eigen_value_gt.round(6)
print 'eigen vectors [numpy]:\n', eigen_vector_gt.round(6)

eigen values  [ours]: [ 137.809125   11.845068    8.51938     8.102546    6.861755]
eigen vectors [ours]:
[[-0.424379 -0.075359  0.862292  0.043971  0.262189]
 [-0.484432 -0.757376 -0.324087  0.294033  0.014764]
 [-0.454031  0.156556  0.05844  -0.238075 -0.84217 ]
 [-0.43071   0.132501 -0.297321 -0.718078  0.4392  ]
 [-0.439968  0.615334 -0.244142  0.58249   0.169977]]
eigen values  [numpy]: [ 137.809125   11.845068    6.861755    8.51938     8.102546]
eigen vectors [numpy]:
[[-0.424379 -0.075359  0.262189 -0.862292  0.043971]
 [-0.484432 -0.757376  0.014764  0.324087  0.294033]
 [-0.454031  0.156556 -0.84217  -0.05844  -0.238075]
 [-0.43071   0.132501  0.4392    0.297321 -0.718078]
 [-0.439968  0.615334  0.169977  0.244142  0.58249 ]]
